In [4]:
import numpy as np
import math

In [18]:
Length_of_imput_sequence, d_k,d_v = 4,8,8
Q = np.random.randn(Length_of_imput_sequence,d_k)
K = np.random.randn(Length_of_imput_sequence,d_k)
V = np.random.randn(Length_of_imput_sequence,d_v)

Self attention = softmax((dot(Q,K)/math.sqrt(d_k)) + M)V

In [19]:
Q_matmul_K = np.matmul(Q,K.T)

In [22]:
Scaled_Q_matmul_K = Q_matmul_K/math.sqrt(d_k) # reduces variance of the Q_matmul_K

## Masking
Masking is needed in the decoder section to not look at the future word when creating the context of the current word
However, Masks can also be used in the encoder section such that the each input token is prevented from getting contect from the tokens that appear later in the sequence

Masking can be emulated by simply using a lower triangular matrix whose values below the diagonal are all 0's and above them is -inf. The reason why we use 0 is because we add this mask to the scaled matmul(Q,K.T) and thereby not affecting its values and -inf to enable softmax

In [34]:
mask = np.tril(np.ones((Length_of_imput_sequence,Length_of_imput_sequence)))

In [35]:
mask[mask == 0] = -np.infty
mask[mask == 1] = 0

In [36]:
mask

array([[  0., -inf, -inf, -inf],
       [  0.,   0., -inf, -inf],
       [  0.,   0.,   0., -inf],
       [  0.,   0.,   0.,   0.]])

In [38]:
Masked_Scaled_Q_matmul_K = (Scaled_Q_matmul_K + mask)

 ## Softmax
 


In [47]:
def softmax(Masked_Scaled_Q_matmul_K):
    return (np.exp(Masked_Scaled_Q_matmul_K).T/np.sum(np.exp(Masked_Scaled_Q_matmul_K),axis = 1)).T

In [48]:
attention = softmax(Masked_Scaled_Q_matmul_K)

In [50]:
attention


array([[1.        , 0.        , 0.        , 0.        ],
       [0.6271699 , 0.3728301 , 0.        , 0.        ],
       [0.19705597, 0.57663204, 0.22631199, 0.        ],
       [0.60900332, 0.07995933, 0.02305373, 0.28798363]])

Notice the effect of using a mask here ^ 

In [51]:
new_v = np.matmul(attention,V)

In [52]:
new_v

array([[ 0.83241343,  0.65783222,  1.02220733, -0.4018901 , -0.24618614,
         0.28513707,  0.9904864 , -0.75213917],
       [ 0.2289083 ,  0.35576851,  0.63442191, -0.36325503, -0.70928762,
         0.03312295,  0.81402309, -1.15149977],
       [-0.10371799,  0.09063954,  0.32839326, -0.04086265, -1.28331618,
        -0.43473941,  0.47172228, -1.24125348],
       [ 0.5463514 ,  0.32616912, -0.04976875, -0.07038777,  0.25460342,
         0.2552575 ,  0.98547674, -0.23249863]])

In [53]:
V

array([[ 0.83241343,  0.65783222,  1.02220733, -0.4018901 , -0.24618614,
         0.28513707,  0.9904864 , -0.75213917],
       [-0.7863001 , -0.15235909, -0.01790564, -0.29826362, -1.48831083,
        -0.39081189,  0.51717882, -1.82329893],
       [ 0.82035336,  0.21591835,  0.60662348,  0.92933901, -1.66405686,
        -1.17348054, -0.09580076, -0.18411479],
       [ 0.28949104, -0.23351355, -2.37808524,  0.61388511,  1.9511479 ,
         0.48582706,  1.19146543,  1.30420922]])

In [58]:
def scaled_dot_product_attention(K,Q,V,mask = None):
    d_k = K.shape[-1]
    scaled = np.matmul(Q,K.T)/math.sqrt(d_k)
    scaled = scaled if mask is None else scaled+mask
    attention = softmax(scaled)
    new_v = np.matmul(attention,V)
    return new_v , attention

In [62]:
new_values,attention = scaled_dot_product_attention(K,Q,V)

In [63]:
new_values

array([[ 0.18124485,  0.155739  ,  0.03281338,  0.0279723 , -0.55665541,
        -0.16453249,  0.67447618, -0.68784406],
       [ 0.35107052,  0.13416902, -0.38009518,  0.18686706,  0.01800877,
        -0.02293615,  0.78384793, -0.16179826],
       [-0.06313566,  0.05718434,  0.04906295,  0.02671258, -0.94949352,
        -0.33972956,  0.54600555, -0.97854129],
       [ 0.5463514 ,  0.32616912, -0.04976875, -0.07038777,  0.25460342,
         0.2552575 ,  0.98547674, -0.23249863]])

# MultiHeaded Attention

breaking down each word vector into N different parts and multiplyting each of them with different Linear Layers with different weights to get a better contextual understanding of the sequence of words 

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F